*Copyright 2021 Abhishek Dutta.*

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Virology Immunology
\begin{align}
    &\dot V = (1-\eta)pV(1-\frac{V}{k_V})-\delta VT-\alpha_v V \\
    &\dot T = s_T+\gamma T(\frac{V}{V+k_T})-\alpha_TT
\end{align}
### Pharmacokinetic Pharmacodynamic
\begin{align}
    &\dot D = -\alpha_DD \\
    &\eta = \frac{D}{D+ec_{50}}
\end{align}
### Toxicity
\begin{align}
    \dot X = D -\alpha_X X \\
\end{align}

In [ ]:
import numpy as np
from scipy import integrate as igt
from scipy import optimize as opt
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp

In [ ]:
def vtd(t, y, dl, dm, n):
    if int(t*10) == rmd[0]*10:
      y[2] = dl
    elif int(t*10) in rmd[1:int(n)]*10:
      y[2] = dm
    V, T, D, X = y
    dV = ec5/(D+ec5)*p*V*(1-V/kv) - dlt*V*T - alp*V
    dT = 0.03e6 + gma*T*V/(V+kt) - 0.03*T
    dD = -3*D
    dX = D - 0.5*X
    return dV, dT, dD, dX

### Derivative Free Optimization

In [ ]:
def cost(vec):
    dl, dm, n = vec
    sol = igt.solve_ivp(fun=vtd, t_span=[0,tf], y0=y0, args=(dl,dm,n), t_eval=pcr, max_step=0.3)
    w = 1e3; error = np.sum(sol.y[0]**2) + w*25*np.sum(sol.y[2]**2)
    return error

In [ ]:
# Viral load lower respiratory tract
lrt = np.array([7.2,6.7,5.2,4.9,4.3,4,2.2,1.2])
vrld = 10 ** lrt
# Viral load upper respiratory tract
urt = np.array([3.65,3.2,2.2,1.9,0.8,0.5,0.25,0.6])
vrld = 10 ** urt
# Viral load recorded after a week at
incb=4; tf=incb+28; pcr = incb+np.array([1,3,5,7,10,14,21,28])
# Remdesivir schedule
rmd = incb+np.arange(1,11,1)
# Initial viral, immmune, drug load
v0=10; t0=1e6; d0=0; x0=0; y0 = [v0,t0,d0,x0]
# Optimized pathogenic parameters upper respiratory
ec5=215; p=5.3; kv=7260; dlt=1e-7; alp=0.87; gma=0.75; kt=840
# Presribed drug dose mg
dl, dm, n = 200, 100, 10
# Global optimization - constrained
ret = opt.differential_evolution(func=cost, bounds=((0,200),(0,100),(1,9)))
dl, dm, n = 92,92,5.5 # ret.x
# Simulate at each day
days = np.arange(0,tf+1,1)
sol = igt.solve_ivp(fun=vtd, t_span=[0,tf], y0=y0, args=(dl,dm,n), max_step=0.1)
sl = igt.solve_ivp(fun=vtd, t_span=[0,tf], y0=y0, args=(200,100,10), max_step=0.1)

In [ ]:
# Create figure with secondary y-axis
fig = sp.make_subplots(rows=2,cols=1,vertical_spacing=0.05,specs=[[{"secondary_y": True}],[{"secondary_y": False}]])
# Add traces
fig.add_trace(go.Scatter(x=sol.t,y=sol.y[0],name='Viral load',line=dict(color='red'),fill='tozeroy'),row=1,col=1,secondary_y=False)
# fig.add_trace(go.Scatter(x=pcr,y=vrld,name='FDA load'))
fig.add_trace(go.Scatter(x=sol.t,y=sol.y[1],name='Immune cell',line=dict(color='green'),fill='tozeroy'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=sol.t,y=sol.y[2],name='Optimized drug',line=dict(color='blue'),fill='tozeroy'),row=2,col=1)
# Set axes titles
fig.update_layout(title='Controlled Remdesivir Schedule for COVID-19')
fig.update_layout(autosize=False,width=10e2,height=5e2,margin=dict(l=5,r=5,t=50,b=5))
fig.update_layout(legend=dict(yanchor='bottom',y=0.25,xanchor='left',x=0.7))
xlim=30; fig.update_xaxes(row=1,col=1,range=[0,xlim])
fig.update_yaxes(title_text='viral copies/ml',row=1,col=1,secondary_y=False)
fig.update_yaxes(title_text='immune cells/ml',row=1,col=1,secondary_y=True)
fig.update_yaxes(title_text='remdesivir mg',row=2,col=1)
fig.update_xaxes(title_text='Days',row=2,col=1,range=[0,xlim])

In [ ]:
area = np.trapz(sol.y[0],sol.t)
print(dl,dm,n,area)

92 92 5.5 26692.307700394216


In [ ]:
# Create figure with secondary y-axis
fig1 = sp.make_subplots(rows=2,cols=1,vertical_spacing=0.05)
# Add traces
fig1.add_trace(go.Scatter(x=sl.t,y=sl.y[3],name='Original Remdesivir administration',line=dict(color='red'),fill='tozeroy'),
               row=1,col=1)
# fig.add_trace(go.Scatter(x=pcr,y=vrld,name='FDA load'))
fig1.add_trace(go.Scatter(x=sol.t,y=sol.y[3],name='Optimized Remdesivir administration',line=dict(color='green'),fill='tozeroy'),
               row=2,col=1)
# Set axes titles
fig1.update_layout(title='Optimized Drug Administration reduces Toxicity')
fig1.update_layout(autosize=False,width=10e2,height=5e2,margin=dict(l=5,r=5,t=50,b=5))
fig1.update_layout(legend=dict(yanchor='bottom',y=0.25,xanchor='left',x=0.7))
ylim = 90; xlim=30; fig1.update_xaxes(row=1,col=1,range=[0,xlim])
fig1.update_yaxes(title_text='toxicity level',row=1,col=1,range=[0,ylim])
fig1.update_yaxes(title_text='toxicity level',row=2,col=1,range=[0,ylim])
fig1.update_xaxes(title_text='Days',row=2,col=1,range=[0,xlim])

In [ ]:
tox1 = np.trapz(sl.y[3],sol.t); tox2 = np.trapz(sol.y[3],sol.t)
print(tox1,tox2)

889.2458445793952 374.0611802299203
